# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

# Normally secrets.py isn't pushed to github
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
# HTTP Request GET Endpoint: /stock/{symbol}/stats/{stat?}
# Testing Sandbox Base URL: https://sandbox.iexapis.com
# API Call (add stable inbetween and token at the end, don't need {stat?}):
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [4]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# Must use chunks function since IEX cloud limits requests to 100 per second
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [5]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.29,-0.225154,N/A
1,AAL,14.96,-0.31378,N/A
2,AAP,193.96,-0.096778,N/A
3,AAPL,150.57,0.001165,N/A
4,ABBV,158.25,0.346255,N/A
...,...,...,...,...
496,YUM,122.61,0.015292,N/A
497,ZBH,105.99,-0.356503,N/A
498,ZBRA,299.67,-0.471915,N/A
499,ZION,50.69,-0.056108,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [6]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,80.72,2.37753,N/A
1,355,OXY,60.93,1.104528,N/A
2,147,DVN,52.92,1.042332,N/A
3,298,MCK,339.00,0.773877,N/A
4,138,DLTR,173.67,0.716275,N/A
5,42,APA,32.94,0.68585,N/A
6,89,CF,87.85,0.682315,N/A
7,315,MRO,21.71,0.673946,N/A
8,225,HRB,37.92,0.620597,N/A
9,463,VAR,185.31,0.590567,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [7]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('This is not a number!')
        print('Please try again: ')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000
10000


In [8]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = position_size/final_dataframe.loc[i, 'Price']
    
final_dataframe

C:\Users\LELE\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,80.72,2.37753,2.477701
1,355,OXY,60.93,1.104528,3.282455
2,147,DVN,52.92,1.042332,3.779289
3,298,MCK,339.00,0.773877,0.589971
4,138,DLTR,173.67,0.716275,1.151609
5,42,APA,32.94,0.68585,6.071645
6,89,CF,87.85,0.682315,2.276608
7,315,MRO,21.71,0.673946,9.212345
8,225,HRB,37.92,0.620597,5.274262
9,463,VAR,185.31,0.590567,1.079273


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [9]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns), ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.60,N/A,-0.225696,N/A,-0.200069,N/A,-0.08216,N/A,0.000658,N/A,N/A
1,AAL,14.83,N/A,-0.309787,N/A,-0.23859,N/A,-0.26,N/A,0.105025,N/A,N/A
2,AAP,191.20,N/A,-0.095951,N/A,-0.211209,N/A,-0.147486,N/A,0.095541,N/A,N/A
3,AAPL,155.79,N/A,0.001137,N/A,-0.142119,N/A,-0.104524,N/A,0.123596,N/A,N/A
4,ABBV,155.15,N/A,0.343712,N/A,0.127408,N/A,-0.067244,N/A,0.104421,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,118.73,N/A,0.015348,N/A,-0.074054,N/A,-0.045813,N/A,0.062457,N/A,N/A
497,ZBH,107.51,N/A,-0.34885,N/A,-0.202594,N/A,-0.200442,N/A,-0.03286,N/A,N/A
498,ZBRA,303.14,N/A,-0.451547,N/A,-0.47064,N/A,-0.268296,N/A,-0.034166,N/A,N/A
499,ZION,51.95,N/A,-0.056933,N/A,-0.320207,N/A,-0.233365,N/A,-0.064728,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [10]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] is None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0.0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.289421
1      0.173653
2       0.52495
3      0.680639
4      0.954092
         ...   
496    0.708583
497     0.11976
498    0.051896
499    0.578842
500    0.427146
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.441118
1      0.321357
2      0.413174
3      0.572854
4      0.952096
         ...   
496    0.676647
497     0.43513
498    0.031936
499    0.173653
500    0.520958
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.658683
1      0.133733
2      0.439122
3      0.590818
4      0.712575
         ...   
496    0.768463
497    0.257485
498    0.117764
499    0.185629
500    0.646707
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.508982
1      0.936128
2      0.906188
3      0.964072
4      0.934132
         ...   
496    0.798403
497    0.313373
498    0.303393
499    0.177645
500    0.872255
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.60,N/A,-0.225696,0.289421,-0.200069,0.441118,-0.08216,0.658683,0.000658,0.508982,N/A
1,AAL,14.83,N/A,-0.309787,0.173653,-0.23859,0.321357,-0.26,0.133733,0.105025,0.936128,N/A
2,AAP,191.20,N/A,-0.095951,0.52495,-0.211209,0.413174,-0.147486,0.439122,0.095541,0.906188,N/A
3,AAPL,155.79,N/A,0.001137,0.680639,-0.142119,0.572854,-0.104524,0.590818,0.123596,0.964072,N/A
4,ABBV,155.15,N/A,0.343712,0.954092,0.127408,0.952096,-0.067244,0.712575,0.104421,0.934132,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,118.73,N/A,0.015348,0.708583,-0.074054,0.676647,-0.045813,0.768463,0.062457,0.798403,N/A
497,ZBH,107.51,N/A,-0.34885,0.11976,-0.202594,0.43513,-0.200442,0.257485,-0.03286,0.313373,N/A
498,ZBRA,303.14,N/A,-0.451547,0.051896,-0.47064,0.031936,-0.268296,0.117764,-0.034166,0.303393,N/A
499,ZION,51.95,N/A,-0.056933,0.578842,-0.320207,0.173653,-0.233365,0.185629,-0.064728,0.177645,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [11]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.60,N/A,-0.225696,0.289421,-0.200069,0.441118,-0.08216,0.658683,0.000658,0.508982,0.474551
1,AAL,14.83,N/A,-0.309787,0.173653,-0.23859,0.321357,-0.26,0.133733,0.105025,0.936128,0.391218
2,AAP,191.20,N/A,-0.095951,0.52495,-0.211209,0.413174,-0.147486,0.439122,0.095541,0.906188,0.570858
3,AAPL,155.79,N/A,0.001137,0.680639,-0.142119,0.572854,-0.104524,0.590818,0.123596,0.964072,0.702096
4,ABBV,155.15,N/A,0.343712,0.954092,0.127408,0.952096,-0.067244,0.712575,0.104421,0.934132,0.888224
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,118.73,N/A,0.015348,0.708583,-0.074054,0.676647,-0.045813,0.768463,0.062457,0.798403,0.738024
497,ZBH,107.51,N/A,-0.34885,0.11976,-0.202594,0.43513,-0.200442,0.257485,-0.03286,0.313373,0.281437
498,ZBRA,303.14,N/A,-0.451547,0.051896,-0.47064,0.031936,-0.268296,0.117764,-0.034166,0.303393,0.126248
499,ZION,51.95,N/A,-0.056933,0.578842,-0.320207,0.173653,-0.233365,0.185629,-0.064728,0.177645,0.278942


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [12]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,107,COG,22.32,N/A,0.345174,0.956088,0.182007,0.972056,0.268154,0.996008,0.259008,1.0,0.981038
1,281,LLY,344.30,N/A,0.411363,0.96008,0.360743,0.994012,0.090007,0.98004,0.130144,0.974052,0.977046
2,273,LB,81.95,N/A,2.342272,1.0,0.852971,1.0,0.224417,0.994012,0.081024,0.860279,0.963573
3,314,MRK,96.61,N/A,0.258033,0.926148,0.178847,0.97006,0.091306,0.982036,0.114467,0.948104,0.956587
4,199,GIS,76.66,N/A,0.31057,0.946108,0.086297,0.924152,0.061776,0.966068,0.136702,0.984032,0.95509
5,225,HRB,37.83,N/A,0.602865,0.984032,0.696381,0.998004,0.397552,1.0,0.069401,0.820359,0.950599
6,470,VRTX,297.06,N/A,0.450071,0.968064,0.232149,0.984032,-0.006492,0.858283,0.155662,0.986028,0.949102
7,428,TAP,61.32,N/A,0.167545,0.888224,0.147996,0.958084,0.055376,0.96008,0.13597,0.982036,0.947106
8,298,MCK,336.00,N/A,0.763022,0.994012,0.287515,0.988024,0.010257,0.918164,0.06599,0.810379,0.927645
9,138,DLTR,171.01,N/A,0.722913,0.992016,0.29864,0.99002,-0.025571,0.828343,0.085108,0.874251,0.921158


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [13]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [14]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\LELE\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,107,COG,22.32,896,0.345174,0.956088,0.182007,0.972056,0.268154,0.996008,0.259008,1.0,0.981038
1,281,LLY,344.30,58,0.411363,0.96008,0.360743,0.994012,0.090007,0.98004,0.130144,0.974052,0.977046
2,273,LB,81.95,244,2.342272,1.0,0.852971,1.0,0.224417,0.994012,0.081024,0.860279,0.963573
3,314,MRK,96.61,207,0.258033,0.926148,0.178847,0.97006,0.091306,0.982036,0.114467,0.948104,0.956587
4,199,GIS,76.66,260,0.31057,0.946108,0.086297,0.924152,0.061776,0.966068,0.136702,0.984032,0.95509
5,225,HRB,37.83,528,0.602865,0.984032,0.696381,0.998004,0.397552,1.0,0.069401,0.820359,0.950599
6,470,VRTX,297.06,67,0.450071,0.968064,0.232149,0.984032,-0.006492,0.858283,0.155662,0.986028,0.949102
7,428,TAP,61.32,326,0.167545,0.888224,0.147996,0.958084,0.055376,0.96008,0.13597,0.982036,0.947106
8,298,MCK,336.00,59,0.763022,0.994012,0.287515,0.988024,0.010257,0.918164,0.06599,0.810379,0.927645
9,138,DLTR,171.01,116,0.722913,0.992016,0.29864,0.99002,-0.025571,0.828343,0.085108,0.874251,0.921158


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: